### Project 3

## Part of the Quantum Computing Bootcamp at the Erdos Institute.

The goal of this project is to implement a quantum calculator for $d$-qubit numbers using qiskit.
There are two main objectives:
1. Implement addition modulo $2^d$;
2. Implement multiplication modulo $2^d$.

These objectives are subject to the following constraints:
- Two $d$-qubit registers, $|x\rangle$ and $|y\rangle$, holding the two input numbers;
- One control qubit, $|z \rangle$, that implements addition if $z = 0$ and multiplication if $z = 1$;
- One $d$-qubit output register, which holds the result of the chosen operation;
- No gates other than arbitrary 1-qubit, control not, and Toffoli gates may be used

Finally, an analysis of the complexities and resources used is to be added at the end of the project.

---

Satisfying the above objectives and constraints would complete the project, but to make our calculator more useful, we would like to implement more operations.
As such, we list the following in order of implementation, where a strikethrough indicates that the operation was added successfully:
- Implement subtraction modulo $2^d$
- Implement modular exponentiation modulo $2^d$


In [349]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator

import numpy as np

### Parameters go here
d = 2

### Load registers

x_register = QuantumRegister(size=d, name="x")
y_register = QuantumRegister(size=d, name="y")
z_register = QuantumRegister(size=1, name="z")
output_register = QuantumRegister(size=d, name="w")

## Circuits necessary for the calculator below here
---
As we are only allowed CX and Toffoli gates for dealing with more than one qubit, we need to decompose the controlled phase gates in the quantum fourier transform in terms of phase gates on one qubit and controlled not gates.

### Complexity & Resources

Each controlled phase gate implementation uses the following:
- 2 CX gates
- 3 Phase gates

It also has depth 4.

In [350]:
### Create circuits that will be necessary to build the calculator
def controlled_phase(register, theta):
    # Creates a controlled phase gate using phase gates and cnot
    # Inputs are a quantum register and a theta angle
    # Always uses first two qubits in register
    quantum_circuit = QuantumCircuit(register, name = f"CP({theta})")
    quantum_circuit.cx(register[0], register[1])
    quantum_circuit.p(- theta / 2, register[1])
    quantum_circuit.cx(register[0],register[1])
    quantum_circuit.p(theta / 2, register[0])
    quantum_circuit.p(theta / 2, register[1])

    return quantum_circuit

#controlled_phase(x_register, np.pi/2).draw()

We need a swap gate decomposed into controlled nots for the same reasoning

### Complexity & Resources

Each swap  gate implementation uses the following:
- 3 CX gates

It also has depth 2.

In [351]:
def swap(register):
    # Creates a swap gate using cnot gates
    # Inputs are a quantum register
    # Always uses first two qubits in register
    quantum_circuit = QuantumCircuit(register, name = "SWAP")    
    quantum_circuit.cx(register[0], register[1])
    quantum_circuit.cx(register[1], register[0])
    quantum_circuit.cx(register[0], register[1])

    return quantum_circuit

#swap(x_register).draw()

We next implement a quantum fourier transform (QFT) and its inverse in much the same way as the second problem session.
Notably, I have added a flag for whether the (QFT) swaps the qubits around after the computation.
This is to save CX gates when computing the QFT followed by its inverse.

### Complexity & Resources

Each QFT implementation on an $n$-qubit register uses the following:
- $n$ Hadamard gates
- $\frac{n^2-n}{2}$ controlled phase gates

When decomposed into just CX and 1-qubit gates, we get:
- $n$ Hadamard gates
- $\frac{3n^2-3n}{2}$ phase gates
- $n^2-n$ CX gates

Finally, the depth is $n$, with the inverse QFT having similar complexity and resources.

In [352]:
def quantum_fourier_transform(register):
    # Creates the quantum fourier transform on a given register
    # Inputs are a quantum register and a default boolean swap
    # needSwap set to True adds swap gates to the result to get the correct order
    # If we are immediately doing the inverse circuit, then it is an unnecessary swap
    # Saves gates to set needSwap = False

    n = len(register) # First get the number of qubits
    quantum_circuit = QuantumCircuit(register, name = "QFT")

    for j, qubit in enumerate(register):
        quantum_circuit.h(qubit)
        for k in range(j+1,n):
            theta = np.pi / (2**(k-j))
            phase = controlled_phase([register[k], qubit], theta).to_gate()
            quantum_circuit.compose(phase, qubits=[k,j],inplace=True)


    return quantum_circuit

def inverse_quantum_fourier_transform(register, needSwap = True):
    # Creates the inverse quantum fourier transform on a given register
    # Inputs are a quantum register and a default boolean swap
    # needSwap set to True adds swap gates to the initial register to get the correct order
    # If we are immediately following the forward circuit, then it is an unnecessary swap
    # Saves gates to set needSwap = False
    n = len(register) # First get the number of qubits
    quantum_circuit = QuantumCircuit(register, name = "IQFT")
    
    for j, qubit in enumerate(register[::-1]):
        for k in range(n,n-j,-1):
            theta = -np.pi / (2**(k+j-n))
            phase = controlled_phase([register[k-1], qubit], theta)
            quantum_circuit.compose(phase, qubits=[k-1,qubit],inplace=True)
        
        quantum_circuit.h(qubit)

    return quantum_circuit

Now that we have our necessary components, we can begin by implementing an adder.
The design for this adder comes from Thomas G. Draper's adder using the QFT, [first produced in 1998](https://arxiv.org/pdf/quant-ph/0008033).
The way it works is through the transform addition circuit implemented below.
Once the QFT is applied to the qubits representing one of our numbers, we perform phase rotations on the result controlled by the other number.
This is the same as adding before the QFT.
We then apply the inverse QFT to discover our result.

### Complexity & Resources

Each Transform Adder implementation on two $n$-qubit registers uses the following:
- $\frac{n^2+n}{2}$ controlled phase gates

When decomposed into just CX and 1-qubit gates, we get:
- $\frac{3n^2+3n}{2}$ phase gates
- $n^2+n $ CX gates

Finally, the depth is $n$.

In [353]:
def transform_adder(*summands):
    # Creates the transform adder circuit described in Draper's paper
    # Takes as input summands, which is a list of two registers
    # The first summand is added to the second in place
    # The second summand needs to be the result of a prior QFT for this to work
    a_register = summands[0]
    b_register = summands[1]

    assert(len(a_register) == len(b_register))
    n = len(a_register)

    quantum_circuit = QuantumCircuit(a_register, b_register, name="TADD")
    
    for j, b in enumerate(b_register):
        for k, a in enumerate(a_register[j:]):
            theta = np.pi / (2**(k))
            phase = controlled_phase([a,b], theta)
            quantum_circuit.compose(phase, qubits=[a,b], inplace=True)

    return quantum_circuit

We are now ready to implement our first addition circuit.

### Complexity & Resources

Each Modulo Adder implementation on two $n$-qubit registers uses the following:
- 1 QFT
- 1 Transform Adder
- 1 Inverse QFT

When decomposed into just CX and 1-qubit gates, we get:
- $\frac{9n^2-3n}{2}$ phase gates
- $2n$ Hadamard gates
- $3n^2 - n$ CX gates

Finally, the depth is $3n$.

In [354]:
def modulo_addition(a_register, b_register):
    # Implements addition modulo $2^d$, where $d$ was defined in our parameters
    # Takes in two registers as summands and their sum is written to the second register
    quantum_circuit = QuantumCircuit(a_register, b_register, name="ADDM")
    assert(len(a_register) == len(b_register))

    QFT = quantum_fourier_transform(b_register).to_gate()
    quantum_circuit.compose(QFT, qubits=b_register, inplace=True)

    TADD = transform_adder(a_register, b_register).to_gate()
    quantum_circuit.compose(TADD, inplace=True)
    
    IQFT = inverse_quantum_fourier_transform(b_register).to_gate()
    quantum_circuit.compose(IQFT, qubits=b_register, inplace=True)

    return quantum_circuit

Before adding this to the full calculator circuit, we must first build two essentially bookkeeping circuits.
As Qiskit encodes its vectors with little-endian ordering, I need to convert little-endian to big-endian for my own ease.
The copy circuit just allows me to not modify the numbers in place.

### Complexity and Resources

The copy circuit on $n$-qubits is relatively simple when copying into an initial 0 state, using only:
- $n$ CX gates

The reverse endian circuit is a little more complicated, but it uses only:
- $3n$ CX gates

It has depth of $3n$ and the copy circuit has depth $n$.

In [355]:
def copy_register(in_register, output):
    # Copies the given input to an output initialized to 0
    # Takes in two registers
    quantum_circuit = QuantumCircuit(in_register, output, name = "COPY")
    
    for j, x in enumerate(in_register):
        quantum_circuit.cx(x,output[j])

    return quantum_circuit

def reverse_endian(register):
    # Reverses the endian of how Qiskit handles states
    # I need this because I spent way too long troubleshooting before I found that the default is little endian
    quantum_circuit = QuantumCircuit(register, name = "REND")
    n = len(register)

    h = n // 2
    offset = n % 2

    for p, q in zip(register[:h], reversed(register[h+offset:])):
        quantum_circuit.compose(swap([p,q]), qubits=[p,q], inplace=True)

    return quantum_circuit

In [356]:
def QCalc(x_register, y_register, z_register, output_register):
    # Implements the calculator, at least the addition part

    quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register, name = "QCALC")
    register = quantum_circuit.qubits

    REND = reverse_endian(register).to_gate()
    quantum_circuit.compose(REND, register, inplace=True)

    COPY = copy_register(x_register, output_register).to_gate()
    quantum_circuit.compose(COPY, qubits= list(x_register) + list(output_register), inplace=True)

    MADD = modulo_addition(y_register, output_register).to_gate()
    quantum_circuit.compose(MADD, qubits= list(y_register) + list(output_register), inplace=True)
    
    quantum_circuit.compose(REND, register, inplace=True)

    return quantum_circuit

mod = 2**d
dims = 2**((d**3)+1)
QCALC = QCalc(x_register, y_register, z_register, output_register)

for a in range(mod):
    for b in range(mod):
        for z in range(2):
            start = a*(mod**d)*2 + b*mod*2 + z*mod
            startState = Statevector.from_int(start, dims)
            end = a*(mod**d)*2 + b*mod*2 + z*mod + ((a+b) % mod)
            endState = Statevector.from_int(end, dims)
            state = startState.evolve(QCALC)

            if state.equiv(endState):
                print("Worked!")
            else:
                print("Something Broke")

QCALC.draw()

Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!
Worked!


┌───────┐┌───────┐         ┌───────┐
x_0: ┤0      ├┤0      ├─────────┤0      ├
     │       ││       │         │       │
x_1: ┤1      ├┤1      ├─────────┤1      ├
     │       ││       │┌───────┐│       │
y_0: ┤2      ├┤       ├┤0      ├┤2      ├
     │       ││       ││       ││       │
y_1: ┤3 REND ├┤  COPY ├┤1      ├┤3 REND ├
     │       ││       ││       ││       │
  z: ┤4      ├┤       ├┤  ADDM ├┤4      ├
     │       ││       ││       ││       │
w_0: ┤5      ├┤2      ├┤2      ├┤5      ├
     │       ││       ││       ││       │
w_1: ┤6      ├┤3      ├┤3      ├┤6      ├
     └───────┘└───────┘└───────┘└───────┘